In [1]:
# Databricks Notebook: silver_processing.py
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [2]:
# === STEP 1: READ AIRLINES DATA ===
airlines_df = spark.read.table("unikargo_dev.01_bronze.unikargo_airlines_bronze")

In [3]:
dim_airline = airlines_df \
    .filter(col("iata_code").isNotNull()) \
    .dropDuplicates(["iata_code"]) \
    .select("iata_code", "airline") \
    .withColumn("airline_sk", row_number().over(Window.orderBy("iata_code"))) \
    .select("airline_sk", "iata_code", "airline")

In [ ]:
# Write the Delta table
dim_airline.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("`unikargo_dev`.`02_silver`.`unikargo_dim_airline_silver`")